In [1]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [2]:
import numpy as np

In [3]:
client = RemoteAPIClient()
sim = client.require('sim')
sim.setStepping(True)


0

In [8]:
# Function the invert a matrix
def invertMatrix(M):
    return np.linalg.inv(M)

In [4]:
# Get Object Position and Orientation
def getPositionAndOrientation(handle):
    pos = sim.getObjectPosition(handle)
    ori = sim.getObjectOrientation(handle)
    return pos, ori

In [5]:
# Get Rotation Matrix fixed on Z axis
def getRotationMatrixZfixed(pos, teta):
    rotationM = np.zeros((3,3))
    rotationM[2][2] = 1
    rotationM[0][0] = np.cos(teta)
    rotationM[0][1] = - np.sin(teta)
    rotationM[1][0] = np.sin(teta)
    rotationM[1][1] = np.cos(teta)

    return rotationM

In [14]:
# get the transformation from local axis to global axis

def getTransformationMatrixLocal2Global(handle):
    pos, ori = getPositionAndOrientation(handle)
    rotM = getRotationMatrixZfixed(pos, ori[2])
    transM = pos 
    scale = np.array([0, 0, 0, 1])

    # Local to Global
    M_L2G = np.column_stack((rotM, transM))
    M_L2G = np.row_stack((M_L2G, scale))

    return M_L2G



In [15]:
# Get the tranfortion from object to another object

def getTransformationMatrixObject2Object(sourceObject, desObject):
    M_S2G = getTransformationMatrixLocal2Global(sourceObject)
    M_G2S = invertMatrix(M_S2G)
    M_D2G = getTransformationMatrixLocal2Global(desObject)
    return M_G2S @ M_D2G

In [17]:
sim.startSimulation()

# Get Objects Handle
personHandle = sim.getObject("/Rapha")
chairHandle = sim.getObject("/Chair")
treeHandle = sim.getObject("/Tree")
diningTableHandle = sim.getObject("/diningTable")
rackHandle = sim.getObject("/Rack")
robotHanle = sim.getObject("/Robot")

# Get transformation Matriz from local to global
result = getTransformationMatrixObject2Object(chairHandle, robotHanle)
print(result)



sim.stopSimulation()

[[-0.94114251 -0.33801002  0.         10.94594603]
 [ 0.33801002 -0.94114251  0.         -1.39993305]
 [ 0.          0.          1.          0.29563239]
 [ 0.          0.          0.          1.        ]]
